<a target="_blank" href="https://colab.research.google.com/github/Aaronau667/FinRL-Tutorials/blob/master/1-Introduction/Stock_NeurIPS2018_ElegantRL.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version**



# Content

* [1. Task Description](#0)
* [2. Install Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. A List of Python Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download and Preprocess Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5. Build Market Environment in OpenAI Gym-style](#4)  
    * [5.1. Data Split](#4.1)  
    * [5.3. Environment for Training](#4.2)    
* [6. Train DRL Agents](#5)
* [7. Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
  

<a id='0'></a>
# Part 1. Task Discription

We train a DRL agent for stock trading. This task is modeled as a Markov Decision Process (MDP), and the objective function is maximizing (expected) cumulative return.

We specify the state-action-reward as follows:

* **State s**: The state space represents an agent's perception of the market environment. Just like a human trader analyzing various information, here our agent passively observes many features and learns by interacting with the market environment (usually by replaying historical data).

* **Action a**: The action space includes allowed actions that an agent can take at each state. For example, a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying. When an action operates multiple shares, a ∈{−k, ..., −1, 0, 1, ..., k}, e.g.. "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* **Reward function r(s, a, s′)**: Reward is an incentive for an agent to learn a better policy. For example, it can be the change of the portfolio value when taking a at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio values at state s′ and s, respectively


**Market environment**: 30 consituent stocks of Dow Jones Industrial Average (DJIA) index. Accessed at the starting date of the testing period.


The data for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Install Python Packages

<a id='1.1'></a>
## 2.1. Install packages


In [25]:
## install finrl library
!pip install wrds
!pip install swig
!pip install ccxt==1.66.1
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git
!pip install pyportfolioopt
!pip install pandas-market-calendars
!pip install zipline-reloaded
## instal elegantrl
!pip install elegantrl



  Using cached ccxt-1.66.1-py2.py3-none-any.whl.metadata (113 kB)
  Using cached yarl-1.7.2.tar.gz (168 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of aiohttp to determine which version is compatible with other requirements. This could take a while.
  Using cached aiohttp-3.11.18-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiohttp-3.11.16-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiohttp-3.11.12-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiohttp-3.11.11-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiohttp-3.11.10-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiohttp-3.11.9-cp311-cp311-manylinux_2_17_x86_64.manylinux


<a id='1.2'></a>
## 2.2. A list of Python packages
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [26]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.elegantrl.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

# from elegantrl.agent import AgentDDPG
# from elegantrl.agent import AgentPPO
# from elegantrl.agent import AgentTD3
# from elegantrl.agent import AgentSAC

import sys
sys.path.append("../FinRL")

import itertools

<a id='1.4'></a>
## 2.4. Create Folders

In [27]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



<a id='2'></a>
# Part 3. Download Data
Yahoo Finance provides stock data, financial news, financial reports, etc. Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** in FinRL-Meta to fetch data via Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).



-----
class YahooDownloader:
    Retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()


In [28]:
# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

'2020-07-31'

In [29]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-31'


In [30]:
import os
import pandas as pd
from datetime import datetime
import time
import random
import requests

def download_and_save_data(start_date, end_date, ticker_list, data_dir='./data'):

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)


    file_name = f"dow30_data_{start_date}_{end_date}.csv"
    file_path = os.path.join(data_dir, file_name)


    if os.path.exists(file_path):
        print(f"Loading existing data from {file_path}")
        return pd.read_csv(file_path, index_col=0, parse_dates=True)

    print("Downloading data from Yahoo Finance...")

    def download_with_retry(ticker, start_date, end_date, max_retries=3, delay=2):
        for attempt in range(max_retries):
            try:
                data = YahooDownloader(start_date=start_date,
                                     end_date=end_date,
                                     ticker_list=[ticker]).fetch_data()
                return data
            except Exception as e:
                if attempt < max_retries - 1:
                    sleep_time = delay + random.uniform(0, 1)
                    print(f"Download failed for {ticker}, retrying in {sleep_time:.2f} seconds...")
                    time.sleep(sleep_time)
                else:
                    print(f"Failed to download {ticker} after {max_retries} attempts")
                    raise e


    all_data = []
    for ticker in ticker_list:
        try:
            data = download_with_retry(ticker, start_date, end_date)
            all_data.append(data)
            print(f"Successfully downloaded {ticker}")
        except Exception as e:
            print(f"Error downloading {ticker}: {str(e)}")


    if all_data:
        final_df = pd.concat(all_data, axis=0)

        final_df.to_csv(file_path)
        print(f"Data saved to {file_path}")
        return final_df
    else:
        raise Exception("No data was downloaded successfully")


df = download_and_save_data(
    start_date=TRAIN_START_DATE,
    end_date=TRADE_END_DATE,
    ticker_list=config_tickers.DOW_30_TICKER
)

Loading existing data from ./data/dow30_data_2009-01-01_2021-10-31.csv


<ipython-input-30-655f54e75f4d>:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.read_csv(file_path, index_col=0, parse_dates=True)


In [31]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [32]:
df.shape

(94331, 8)

In [33]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,close,high,low,open,volume,tic,day
0,2009-01-02,2.727418,2.736134,2.559416,2.581054,746015200,AAPL,4
1,2009-01-02,40.791458,40.853693,39.934000,40.514858,6547900,AMGN,4
2,2009-01-02,14.929290,15.076034,14.211015,14.342313,10955700,AXP,4
3,2009-01-02,33.941101,34.173627,32.088404,32.103406,7010200,BA,4
4,2009-01-02,30.344677,30.389958,28.921562,29.050937,7117200,CAT,4


# Part 4: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

In [34]:
import os

# Create data directory if it doesn't exist
if not os.path.exists("data"):
    os.makedirs("data")


processed_file_path = f"data/dow30_data_{TRAIN_START_DATE}_{TRADE_END_DATE}_processed.csv"

# Check if processed data already exists
if os.path.exists(processed_file_path):
    print(f"Loading existing processed data from {processed_file_path}")
    processed = pd.read_csv(processed_file_path)
else:
    print("Processing data...")
    fe = FeatureEngineer(
                        use_technical_indicator=True,
                        tech_indicator_list = INDICATORS,
                        use_vix=True,
                        use_turbulence=True,
                        user_defined_feature = False)

    processed = fe.preprocess_data(df)

    # Save processed data to data directory
    processed.to_csv(processed_file_path, index=False)
    print(f"Processed data saved to {processed_file_path}")

Loading existing processed data from data/dow30_data_2009-01-01_2021-10-31_processed.csv


In [35]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [36]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,2.727418,2.736134,2.559416,2.581054,746015200.0,4.0,0.0,2.947757,2.622185,100.0,66.666667,100.0,2.727418,2.727418,39.189999,0.0
1,2009-01-02,AMGN,40.791458,40.853693,39.934000,40.514858,6547900.0,4.0,0.0,2.947757,2.622185,100.0,66.666667,100.0,40.791458,40.791458,39.189999,0.0
2,2009-01-02,AXP,14.929290,15.076034,14.211015,14.342313,10955700.0,4.0,0.0,2.947757,2.622185,100.0,66.666667,100.0,14.929290,14.929290,39.189999,0.0
3,2009-01-02,BA,33.941101,34.173627,32.088404,32.103406,7010200.0,4.0,0.0,2.947757,2.622185,100.0,66.666667,100.0,33.941101,33.941101,39.189999,0.0
4,2009-01-02,CAT,30.344677,30.389958,28.921562,29.050937,7117200.0,4.0,0.0,2.947757,2.622185,100.0,66.666667,100.0,30.344677,30.344677,39.189999,0.0
5,2009-01-02,CRM,8.444491,8.489171,7.856207,7.967906,4069200.0,4.0,0.0,2.947757,2.622185,100.0,66.666667,100.0,8.444491,8.444491,39.189999,0.0
6,2009-01-02,CSCO,11.166082,11.192418,10.698635,10.803975,40980600.0,4.0,0.0,2.947757,2.622185,100.0,66.666667,100.0,11.166082,11.166082,39.189999,0.0
7,2009-01-02,CVX,39.716721,40.121573,38.190754,38.528129,13695900.0,4.0,0.0,2.947757,2.622185,100.0,66.666667,100.0,39.716721,39.716721,39.189999,0.0
8,2009-01-02,DIS,20.346144,20.439709,19.138304,19.359458,9796600.0,4.0,0.0,2.947757,2.622185,100.0,66.666667,100.0,20.346144,20.346144,39.189999,0.0
9,2009-01-02,GS,65.680351,66.331402,62.220702,63.606071,14088500.0,4.0,0.0,2.947757,2.622185,100.0,66.666667,100.0,65.680351,65.680351,39.189999,0.0


<a id='4'></a>
# Part 5. Build A Market Environment in OpenAI Gym-style
The training process involves observing stock price change, taking an action and reward's calculation. By interacting with the market environment, the agent will eventually derive a trading strategy that may maximize (expected) rewards.

Our market environment, based on OpenAI Gym, simulates stock markets with historical market data.

## Data Split
We split the data into training set and testing set as follows:

Training data period: 2009-01-01 to 2020-07-01

Trading data period: 2020-07-01 to 2021-10-31


In [37]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))



83897
9744


In [38]:
train.tail()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
2892,2020-06-30,UNH,275.247681,276.647482,268.444637,269.293853,2932900.0,1.0,-0.018634,290.693741,259.441665,52.413029,-20.026992,0.598601,275.481010,268.767363,30.43,12.918869
2892,2020-06-30,V,186.425079,186.984829,183.520185,184.803747,9040100.0,1.0,1.025075,194.257207,180.858032,53.021032,-51.428693,2.103894,187.155981,177.570353,30.43,12.918869
2892,2020-06-30,VZ,41.323433,41.443363,40.746269,41.166022,17414800.0,1.0,-0.358556,44.228628,39.972071,48.097057,-50.671001,8.321180,41.844624,42.215851,30.43,12.918869
2892,2020-06-30,WBA,32.880199,33.027577,32.391533,32.670771,4782100.0,1.0,-0.070742,35.890254,30.733451,48.830192,-14.266623,0.948620,32.963970,32.795457,30.43,12.918869
2892,2020-06-30,WMT,37.177689,37.286321,36.792813,37.003876,20509200.0,1.0,-0.283845,38.250862,36.341647,48.159682,-69.838555,3.558157,37.711031,38.330750,30.43,12.918869


In [39]:
trade.head()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2020-07-01,AAPL,88.485008,89.274814,88.436409,88.730458,110737200.0,2.0,2.967002,91.235594,78.913921,62.807122,107.472188,29.811308,82.608456,76.490387,28.620001,53.06822
0,2020-07-01,AMGN,218.336441,219.286413,199.046295,201.562404,6575800.0,2.0,3.306037,211.284559,182.023957,61.279633,272.795330,47.010004,195.339151,196.314673,28.620001,53.06822
0,2020-07-01,AXP,88.493980,91.232068,88.108198,89.623087,3301000.0,2.0,-0.373984,106.116189,84.335733,48.504812,-62.638343,1.752213,93.451014,87.157390,28.620001,53.06822
0,2020-07-01,BA,180.320007,190.610001,180.039993,185.880005,49036700.0,2.0,5.443193,220.721139,160.932863,50.925771,24.220608,15.936440,176.472335,155.614168,28.620001,53.06822
0,2020-07-01,CAT,113.824562,116.840377,113.662033,116.822328,2807800.0,2.0,1.200618,123.232892,106.939048,52.865423,35.366408,14.542205,112.655857,107.215975,28.620001,53.06822


In [40]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [41]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 29, State Space: 291


In [42]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)


## Environment for Training



In [43]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Train DRL Agents
* The DRL algorithms are from **Stable Baselines 3**. Users are also encouraged to try **ElegantRL** and **Ray RLlib**.
* FinRL includes fine-tuned standard DRL algorithms, such as DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [44]:
price_array = processed['close'].values
tech_array = processed[INDICATORS].values
turbulence_array = processed['turbulence'].values
agent = DRLAgent(env=env_train, price_array=price_array, tech_array=tech_array, turbulence_array=turbulence_array)

if_using_a2c = False
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = False
if_using_sac = True

### Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


In [45]:
import os

cpu_count = os.cpu_count()
print(f"Colab 分配的 CPU 线程数: {cpu_count}")

Colab 分配的 CPU 线程数: 2


### Agent 1: A2C


In [46]:
import numpy as np

# Pivot close prices to get a 2D array: rows are dates, columns are tickers
price_array = processed.pivot(index="date", columns="tic", values="close").values

# Pivot technical indicators: stack all indicators for all stocks
tech_list = []
for indicator in INDICATORS:
    tech_list.append(processed.pivot(index="date", columns="tic", values=indicator).values)
# Stack along the last axis and flatten stocks and indicators
tech_array = np.stack(tech_list, axis=2)  # shape: (dates, stocks, indicators)
tech_array = tech_array.reshape(tech_array.shape[0], -1)  # shape: (dates, stocks*indicators)

# Pivot turbulence: one value per date
turbulence_array = processed.groupby("date")["turbulence"].first().values

# Now initialize DRLAgent with the correct shapes
agent = DRLAgent(
    env=env_train,
    price_array=price_array,
    tech_array=tech_array,
    turbulence_array=turbulence_array
)

model_kwargs = {
    "learning_rate": 0.001,        # Increased learning rate for faster convergence
    "buffer_size": 50000,          # Reduced buffer size to decrease memory usage
    "learning_starts": 500,        # Reduced warm-up steps
    "batch_size": 128,             # Increased batch size for better training efficiency
    "tau": 0.01,                   # Increased target network update rate
    "gamma": 0.99,                 # Kept unchanged (discount factor)
    "train_freq": 4,               # Increased training frequency
    "gradient_steps": 4,           # Increased gradient steps per update
    "action_noise": None,          # Kept unchanged
    "optimize_memory_usage": True, # Enabled memory optimization
    "ent_coef": "auto",            # Kept unchanged
    "target_update_interval": 500, # Reduced target network update interval
    "target_policy_noise": 0.1,    # Reduced policy noise
    "target_noise_clip": 0.3,      # Reduced noise clipping range
    "policy_kwargs": {
        "net_arch": [128, 128]     # Simplified network architecture
    },
    "verbose": 1                   # Kept unchanged
}
model_a2c = agent.get_model("a2c", model_kwargs)

In [47]:
trained_a2c = agent.train_model(
    model=model_a2c,
    cwd="results/a2c",
    total_timesteps=50000
) if if_using_a2c else None

### Agent 2: DDPG

In [48]:
# Pivot close prices to get a 2D array: rows are dates, columns are tickers
price_array = processed.pivot(index="date", columns="tic", values="close").values

# Pivot technical indicators: stack all indicators for all stocks
tech_list = []
for indicator in INDICATORS:
    tech_list.append(processed.pivot(index="date", columns="tic", values=indicator).values)
# Stack along the last axis and flatten stocks and indicators
tech_array = np.stack(tech_list, axis=2)  # shape: (dates, stocks, indicators)
tech_array = tech_array.reshape(tech_array.shape[0], -1)  # shape: (dates, stocks*indicators)

# Pivot turbulence: one value per date
turbulence_array = processed.groupby("date")["turbulence"].first().values

# Initialize DRLAgent with the correct shapes
agent = DRLAgent(
    env=env_train,
    price_array=price_array,
    tech_array=tech_array,
    turbulence_array=turbulence_array
)
# Optimized DDPG parameters
model_kwargs = {
    "learning_rate": 0.001,        # Increased learning rate for faster convergence
    "buffer_size": 50000,          # Reduced buffer size to decrease memory usage
    "learning_starts": 500,        # Reduced warm-up steps
    "batch_size": 256,             # Increased batch size for better training efficiency
    "tau": 0.01,                   # Increased target network update rate
    "gamma": 0.99,                 # Kept unchanged (discount factor)
    "train_freq": 4,               # Increased training frequency
    "gradient_steps": 4,           # Increased gradient steps per update
    "action_noise": None,          # Kept unchanged
    "optimize_memory_usage": True, # Enabled memory optimization
    "ent_coef": "auto",            # Kept unchanged
    "target_update_interval": 500, # Reduced target network update interval
    "target_policy_noise": 0.1,    # Reduced policy noise
    "target_noise_clip": 0.3,      # Reduced noise clipping range
    "policy_kwargs": {
        "net_arch": [128, 128]     # Simplified network architecture
    },
    "verbose": 1                   # Kept unchanged
}
# Get DDPG model
model_ddpg = agent.get_model("ddpg", model_kwargs)

if if_using_ddpg:
    # Set up logger directory
    tmp_path = RESULTS_DIR + '/ddpg'
    # Configure logger with stdout, csv, and tensorboard outputs
    new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
    # Set the new logger for the DDPG model
    model_ddpg.set_logger(new_logger_ddpg)

In [49]:
trained_ddpg = agent.train_model(
    model=model_ddpg,
    tb_log_name='ddpg',
    total_timesteps=50000,         # Total training steps
    cwd="results/ddpg"             # Directory to save model and logs
) if if_using_ddpg else None

### Agent 3: PPO

In [50]:
import numpy as np

# Data processing
price_array = processed.pivot(index="date", columns="tic", values="close").values

# Process technical indicators
tech_list = []
for indicator in INDICATORS:
    tech_list.append(processed.pivot(index="date", columns="tic", values=indicator).values)
tech_array = np.stack(tech_list, axis=2)  # shape: (dates, stocks, indicators)
tech_array = tech_array.reshape(tech_array.shape[0], -1)  # shape: (dates, stocks*indicators)

# Process turbulence
turbulence_array = processed.groupby("date")["turbulence"].first().values

# Initialize DRLAgent
agent = DRLAgent(
    env=env_train,
    price_array=price_array,
    tech_array=tech_array,
    turbulence_array=turbulence_array
)

# Optimized PPO parameters
model_kwargs = {
    "learning_rate": 0.001,        # Increased learning rate for faster convergence
    "n_steps": 2048,              # Number of steps to run for each environment per update
    "batch_size": 128,            # Increased batch size for better training efficiency
    "n_epochs": 10,               # Number of epochs when optimizing the surrogate loss
    "gamma": 0.99,                # Discount factor
    "gae_lambda": 0.95,           # Factor for trade-off of bias vs variance for GAE
    "clip_range": 0.2,            # Clipping parameter for PPO
    "clip_range_vf": None,        # Clipping parameter for value function
    "ent_coef": 0.01,             # Entropy coefficient for exploration
    "vf_coef": 0.5,               # Value function coefficient
    "max_grad_norm": 0.5,         # Maximum norm for gradient clipping
    "use_sde": False,             # Whether to use State Dependent Exploration
    "sde_sample_freq": -1,        # Sample a new noise matrix every n steps
    "target_kl": None,            # Target KL divergence between updates
    "tensorboard_log": None,      # Log location for tensorboard
    "create_eval_env": False,     # Whether to create a second environment for evaluation
    "policy_kwargs": {
        "net_arch": [128, 128]    # Simplified network architecture
    },
    "verbose": 1,                 # Verbosity level
    "seed": None,                 # Random seed
    "device": "auto",             # Device to use for training
    "_init_setup_model": True     # Whether to initialize the model
}

# Get PPO model
model_ppo = agent.get_model("ppo", model_kwargs)

In [51]:
# Train PPO model
trained_ppo = agent.train_model(
    model=model_ppo,
    cwd="results/ppo",         # Directory to save model and logs
    total_timesteps=50000      # Total number of timesteps to train
) if if_using_ppo else None    # Only train if if_using_ppo is True

### Agent 4: TD3

In [52]:
# Initialize DRLAgent
agent = DRLAgent(
    env=env_train,
    price_array=price_array,
    tech_array=tech_array,
    turbulence_array=turbulence_array
)

model_kwargs = {
    "learning_rate": 0.001,        # Increased learning rate for faster convergence
    "buffer_size": 50000,          # Reduced buffer size to decrease memory usage
    "learning_starts": 500,        # Reduced warm-up steps
    "batch_size": 256,             # Increased batch size for better training efficiency
    "tau": 0.01,                   # Increased target network update rate
    "gamma": 0.99,                 # Kept unchanged (discount factor)
    "train_freq": 4,               # Increased training frequency
    "gradient_steps": 4,           # Increased gradient steps per update
    "action_noise": None,          # Kept unchanged
    "optimize_memory_usage": True, # Enabled memory optimization
    "ent_coef": "auto",            # Kept unchanged
    "target_update_interval": 500, # Reduced target network update interval
    "target_policy_noise": 0.1,    # Reduced policy noise
    "target_noise_clip": 0.3,      # Reduced noise clipping range
    "policy_delay": 2,             # TD3 specific: policy update delay
    "policy_kwargs": {
        "net_arch": [128, 128]     # Simplified network architecture
    },
    "verbose": 1                   # Kept unchanged
}

# Get TD3 model with optimized parameters
model_td3 = agent.get_model("td3", model_kwargs)

In [53]:
trained_td3 = agent.train_model(
    model=model_td3,
    tb_log_name='td3',
    total_timesteps=50000,         # Total training steps
    cwd="results/td3"              # Directory to save model and logs
) if if_using_td3 else None

### Agent 5: SAC

In [54]:
import numpy as np

# Data processing
price_array = processed.pivot(index="date", columns="tic", values="close").values

# Process technical indicators
tech_list = []
for indicator in INDICATORS:
    tech_list.append(processed.pivot(index="date", columns="tic", values=indicator).values)
tech_array = np.stack(tech_list, axis=2)  # shape: (dates, stocks, indicators)
tech_array = tech_array.reshape(tech_array.shape[0], -1)  # shape: (dates, stocks*indicators)

# Process turbulence
turbulence_array = processed.groupby("date")["turbulence"].first().values

# Initialize DRLAgent
agent = DRLAgent(
    env=env_train,
    price_array=price_array,
    tech_array=tech_array,
    turbulence_array=turbulence_array
)

# Optimized SAC parameters
model_kwargs = {
    "learning_rate": 0.001,        # Increased learning rate for faster convergence
    "buffer_size": 50000,          # Reduced buffer size to decrease memory usage
    "learning_starts": 500,        # Reduced warm-up steps
    "batch_size": 128,             # Increased batch size for better training efficiency
    "tau": 0.01,                   # Target network update rate
    "gamma": 0.99,                 # Discount factor
    "train_freq": 4,               # Increased training frequency
    "gradient_steps": 4,           # Increased gradient steps per update
    "action_noise": None,          # No action noise
    "optimize_memory_usage": True, # Enabled memory optimization
    "ent_coef": "auto",            # Automatic entropy coefficient
    "target_update_interval": 500, # Reduced target network update interval
    "target_entropy": "auto",      # Automatic target entropy
    "use_sde": False,              # No state-dependent exploration
    "sde_sample_freq": -1,         # No SDE sampling
    "use_sde_at_warmup": False,    # No SDE at warmup
    "policy_kwargs": {
        "net_arch": [128, 128]     # Simplified network architecture
    },
    "verbose": 1                   # Verbosity level
}

# Get SAC model
model_sac = agent.get_model("sac", model_kwargs)

# Set training flag
if_using_sac = True  # Set to True to train SAC model



In [ ]:
# Train SAC model
trained_sac = agent.train_model(
    model=model_sac,
    cwd="results/sac",         # Directory to save model and logs
    total_timesteps=50000      # Total number of timesteps to train
) if if_using_sac else None    # Only train if if_using_sac is True

| train_agent_multiprocessing() with GPU_ID 0
| Arguments Remove cwd: results/sac


## In-sample Performance

Assume that the initial capital is $1,000,000.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [ ]:
insample_risk_indicator.vix.describe()

In [ ]:
insample_risk_indicator.vix.quantile(0.996)

In [ ]:
insample_risk_indicator.turbulence.describe()

In [ ]:
insample_risk_indicator.turbulence.quantile(0.996)

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends.

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [ ]:
trade.head()

In [ ]:
trained_moedl = trained_sac
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_moedl,
    environment = e_trade_gym)

In [ ]:
df_account_value.shape

In [ ]:
df_account_value.tail()

In [ ]:
df_actions.head()

<a id='6'></a>
# Part 7: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI",
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


In [ ]:
df_account_value.loc[0,'date']

In [ ]:
df_account_value.loc[len(df_account_value)-1,'date']

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value,
             baseline_ticker = '^DJI',
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])